<a href="https://colab.research.google.com/github/daya448/deduplication/blob/main/Deduplicates_using_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Connect to Elasticsearch

ℹ️ We're using an Elastic Cloud deployment of Elasticsearch for this notebook. If you don't have an Elastic Cloud deployment, sign up [here](https://cloud.elastic.co/registration?onboarding_token=search&utm_source=github&utm_content=elasticsearch-labs-notebook) for a free trial.

We'll use the **Cloud ID** to identify our deployment, because we are using Elastic Cloud deployment. To find the Cloud ID for your deployment, go to https://cloud.elastic.co/deployments and select your deployment.

You will also need your **API KEY** to access your deployment. You can [create a new API key](https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#creating-an-api-key) from the `Stack Management -> API keys` menu in Kibana. Be sure to copy or write down your key in a safe place once it is created it will be displayed only upon creation.

In [ ]:
#@title Prepare Elasticsearch Index

# Installs the Elasticsearch client library if not already installed
!pip install elasticsearch pandas
!pip install -U langchain-community
!pip install -U streamlit
!npm install -g localtunnel

from elasticsearch import Elasticsearch, helpers

# Connects to your Elasticsearch cluster (adjust the URL and authentication as necessary)

# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#finding-your-cloud-id
ELASTIC_CLOUD_ID = getpass("Elastic Cloud ID: ")

# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#creating-an-api-key
ELASTIC_API_KEY = getpass("Elastic API Key: ")

#CLOUD_ID = 'general-purpose:YXAtc291dGhlYXN0LTIuYXdzLmZvdW5kLmlvOjQ0MyRmMDg3MDc0Njk4ZTY0YTQ4OWUwZjAzMDNlNGJkZThhMCQyMmJkMmVkODY2MGQ0NmM2OTI3ZjBmMjY4NGUzNDEzZQ=='
#API_KEY = 'VFlZeWI1SUJCY1BMSUswYjN4MW46dXZwQTVJUlZRdTZEZXFpREFBMy1wZw=='

# Create Elasticsearch client
es = Elasticsearch(
    cloud_id=ELASTIC_CLOUD_ID,
    api_key=ELASTIC_API_KEY
)


# Define the index template
index_template = {
        "index_patterns": [
            "names-*"
        ],
        "settings": {
            "index": {
                "analysis": {
                    "filter": {
                        "my_dmetaphone_filter": {
                            "replace": "false",
                            "type": "phonetic",
                            "encoder": "double_metaphone"
                        },
                        "my_soundex": {
                            "type": "phonetic",
                            "encoder": "soundex"
                        }
                    },
                    "analyzer": {
                        "name_analyzer_soundex": {
                            "filter": [
                                "lowercase",
                                "my_soundex"
                            ],
                            "tokenizer": "standard"
                        },
                        "name_analyzer_dmetaphone": {
                            "filter": [
                                "lowercase",
                                "my_dmetaphone_filter"
                            ],
                            "tokenizer": "standard"
                        }
                    }
                },
                "number_of_shards": "1",
                "number_of_replicas": "1"
            }
        },
        "mappings": {
            "properties": {
                "address": {
                    "type": "text",
                    "fields": {
                        "keyword": {
                            "ignore_above": 256,
                            "type": "keyword"
                        }
                    }
                },
                        "dob": {
                            "type": "text",
                            "fields": {
                                "keyword": {
                                    "ignore_above": 256,
                                    "type": "keyword"
                                }
                            }

                },
                "name": {
                    "analyzer": "name_analyzer_dmetaphone",
                    "type": "text",
                    "fields": {
                        "soundex": {
                            "analyzer": "name_analyzer_soundex",
                            "type": "text"
                        }
                    }
                }
            }
        }
    }

# Add the index template to Elasticsearch
try:
    response = es.indices.put_template(name="name-search", body=index_template)
    print("Index template created successfully:", response)
except Exception as e:
    print("Error creating index template:", e)

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼
changed 22 packages in 2s
⠼
⠼3 packages are looking for funding
⠼  run `npm fund` for details
⠼Index template created successfully: {'acknowledged': True}


<ipython-input-55-0bb981d90e32>:99: ElasticsearchWarning: Legacy index templates are deprecated in favor of composable templates.
  response = es.indices.put_template(name="name-search", body=index_template)


In [ ]:
#@title Upload Names Dataset(csv)

# Import required libraries
import pandas as pd
import json

# Load CSV data from a file (Upload the file first in Colab)
from google.colab import files
uploaded = files.upload()  # This will allow you to upload your CSV file

# Specify the filename here (if only one file is uploaded)
csv_filename = list(uploaded.keys())[0]

# Read the CSV file into a Pandas DataFrame
df = pd.read_csv(csv_filename)

# Check the CSV content (first few rows)
print("CSV Content:")
print(df.head())

# Define the index name where the data will be ingested
INDEX_NAME = 'names-search'

# Function to create bulk data from the DataFrame
def generate_bulk_data(df):
    for index, row in df.iterrows():
        # Create an action for each document
        yield {
            "_index": INDEX_NAME,
            "_source": row.to_dict()  # Convert the row to dictionary
        }
# Check if the index already exists
if es.indices.exists(index=INDEX_NAME):
    # Delete the index if it exists
    es.indices.delete(index=INDEX_NAME)
    print(f"Index '{INDEX_NAME}' deleted.")

# Use the bulk API to ingest all documents
try:
    response = helpers.bulk(es, generate_bulk_data(df))
    print("Data ingestion complete!")
    print(f"Indexed {response[0]} documents successfully.")
except Exception as e:
    print("Error during bulk ingestion:", e)

KeyboardInterrupt: 

In [ ]:
#@title Prepare Service Layer
%%writefile app.py

import streamlit as st
import json
import os
from elasticsearch import Elasticsearch
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import AzureChatOpenAI
from langchain.chains import LLMChain

# Set your Azure OpenAI credentials
os.environ["AZURE_OPENAI_API_KEY"] = "8971cf727130428394cb36204e89fe2f"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://carlo-gpt-au.openai.azure.com/"
os.environ["AZURE_OPENAI_API_VERSION"] = "2024-08-01-preview"  # Adjust to your API version

# Connect to your Elasticsearch cluster (adjust the URL and authentication as necessary)
CLOUD_ID = 'general-purpose:YXAtc291dGhlYXN0LTIuYXdzLmZvdW5kLmlvOjQ0MyRmMDg3MDc0Njk4ZTY0YTQ4OWUwZjAzMDNlNGJkZThhMCQyMmJkMmVkODY2MGQ0NmM2OTI3ZjBmMjY4NGUzNDEzZQ=='
API_KEY = 'VFlZeWI1SUJCY1BMSUswYjN4MW46dXZwQTVJUlZRdTZEZXFpREFBMy1wZw=='

# Create Elasticsearch client
es = Elasticsearch(
    cloud_id=CLOUD_ID,
    api_key=API_KEY
)

# Initialize the AzureChatOpenAI model
model = AzureChatOpenAI(
    openai_api_key=os.environ["AZURE_OPENAI_API_KEY"],
    deployment_name="carlo-gpt4o",
    model="gpt-4o",
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"]
)


# Function to query Elasticsearch
def query_elasticsearch(index,input_name,input_address,size=5):
    response = es.search(index=index, body={
  "query": {
    "bool": {
      "must": [
        {
          "match_phrase": {
            "name": input_name
          }
        },
        {
          "match": {
            "address": input_address
          }
        }
      ]
    }
  },
  "size": size
})
    return response['hits']['hits']

# Function to create the prompt and send to OpenAI API
def check_duplicates(search_name,input_address,response_names):
    # Create prompt template
    prompt_template = PromptTemplate(
        template="""
        Compare the Input Name, Input Address against each response name and address and also generate a match percentage for each.

        Input Name: {search_name}
        Input Address: {input_address}
        Response from Elasticsearch with Name and its address:
        {response_names}

        Please format your response in a table format with columns for Response Name,Response Address, Match Percentage, and Duplicate Status.
        If the Match percentage is above 80% consider it as duplicate.
        Short Explanation of Why it is a match.
        Always use Jaro-Winkler algorithm for name similarity comparison.
        Sort the results in descending order of Match Percentage.
        Turn off preamble and only provide the table.
        """,
        input_variables=["search_name","input_address", "response_names"],
    )

    # Prepare prompt input
    prompt_input = {
        "search_name": search_name,
        "input_address": input_address,
        "response_names": response_names
    }

    # Create the LLM chain
    chain = LLMChain(llm=model, prompt=prompt_template)

    # Get the response from OpenAI
    response = chain.run(prompt_input)
    return response

# Main function for Streamlit app
def main():
    st.set_page_config(page_title="Duplicate Detection", layout="centered")

    # Custom CSS for styling
    st.markdown("""
        <style>
            body { font-family: 'Arial', sans-serif; color: #333; }
            .stTextInput input { background-color: #f0f8ff; padding: 10px; border-radius: 5px; }
            .stButton button { background-color: #4CAF50; color: white; border-radius: 5px; }
            .stButton button:hover { background-color: #45a049; }
            .response-table th, .response-table td { padding: 10px; border: 1px solid #ddd; }
            .response-table th { background-color: #f4f4f4; }
            .response-table td { text-align: center; }
        </style>
    """, unsafe_allow_html=True)

    st.title("🔍 Duplicate Detection")
    st.write("Enter the name and address to search for potential duplicates in the database.")
    # Input field for search name
    input_name = st.text_input("Search Name", placeholder="Enter the name you want to search for...")
    input_address = st.text_input("Enter Address", placeholder="Enter the address")

    if st.button("Search 🔍"):
        if input_name and input_address:
            index_name = "names-search"  # Replace with your index name


            # Query Elasticsearch for potential duplicates
            es_response = query_elasticsearch(index_name,input_name,input_address)

            # Build context from results
            if es_response:
                # Collect the top results for names
                for name_doc in es_response:
                    names = name_doc['_source']['name']
                #names = [name_doc['_source']['name'] for name_doc in es_response]

                # Prepare a formatted string for response names
                response_names = "\\n".join(names)

                # Send the details to OpenAI for further processing
                #response = check_duplicates(search_name, response_names)
                response = check_duplicates(input_name,input_address,es_response)

                # Display the response on the UI
                st.write("### Results Comparision from Dataset")

                # Attempt to parse the response into a structured format (if applicable)
                try:
                    st.markdown(response)
                    response_data = json.loads(response)
                    st.table(response_data)
                    st.markdown(response_data)
                except json.JSONDecodeError:
                    st.write("Used Jaro-Winkler algorithm for name similarity comparison")
            else:
                st.write("❌ No potential duplicates found.")
        else:
            st.write("⚠️ Please enter both the name and address to search.")


if __name__ == "__main__":
    main()

Overwriting app.py


In [ ]:
#@title Search ES and Generate Response - Start UI
# Start the Streamlit app
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 --subdomain deduplication & curl ipv4.icanhazip.com
#!streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py



34.148.112.234
⠙⠹⠸⠼⠴⠦your url is: https://deduplication.loca.lt
/tools/node/lib/node_modules/localtunnel/bin/lt.js:81
    throw err;
    ^

Error: connection refused: localtunnel.me:16271 (check your firewall settings)
    at Socket.<anonymous> (/tools/node/lib/node_modules/localtunnel/lib/TunnelCluster.js:52:11)
    at Socket.emit (node:events:517:28)
    at emitErrorNT (node:internal/streams/destroy:151:8)
    at emitErrorCloseNT (node:internal/streams/destroy:116:3)
    at process.processTicksAndRejections (node:internal/process/task_queues:82:21)

Node.js v18.20.5
⠙